# Toy example for Radix-based Benders

In [1]:
%load_ext line_profiler

In [2]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7f4cd2c6cd88>)


### Try optimizing using radix for one condition first

In [3]:
#----------------------------------------
# Starting from basal model
ijomc = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
keff0 = 1./65/3600
#crowding_bound = 0.0003
crowding_bound = 0.001
not_crowded = ['ATPM']
rxns_c = [r for r in ijomc.reactions if all([m.compartment=='c' for m in r.metabolites.keys()]) and 'BIOMASS' not in r.id and r.id not in not_crowded]
crowding_dict = {rxn:keff0 for rxn in rxns_c}
#----------------------------------------

# Temporarily add crowding constraint for the duality gap constraint
crowding = Constraint('crowding')
crowding._bound = crowding_bound
crowding._constraint_sense = 'L'
for rxn,keff in iteritems(crowding_dict):
    rxn.add_metabolites({crowding:keff})

In [4]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

0.873921506968


In [5]:
opt = Optimizer(ijomc)
gap = opt.add_duality_gap_constraint(INF=1e3)

In [6]:
gap.optimize(solver='gurobi')

<Solution 1.75 at 0x7f4d14b6d790>

In [7]:
import numpy as np

radix = 2.
powers = np.arange(-3,4)
print(powers)
digits_per_power = radix
pwr_max = max(powers)
digits = np.linspace(0, radix-1, digits_per_power)
print(digits)

# Discretize crowding coefficients into radix
crowding_p = gap.metabolites.crowding

var_cons_dict = {}
for rxn_p in crowding_p.reactions:
    # Get the coefficient in the dual
    var_d = gap.reactions.wa_crowding
    cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]
    a0 = rxn_p.metabolites[crowding_p]    
    var_cons_dict[rxn_p.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]

[-3 -2 -1  0  1  2  3]
[0. 1.]


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:8 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [8]:
%lprun -f opt.to_radix opt.to_radix(gap, var_cons_dict, radix, powers, digits_per_power)

In [9]:
# %time opt.to_radix(gap, var_cons_dict, radix, powers, digits_per_power)

In [10]:
for rxn in gap.reactions:
    rxn.objective_coefficient = 0.
for rxn in gap.reactions.query('binary'):
    rxn.objective_coefficient = 1.

In [11]:
N_CONDS = 1

df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')
ex_rxns = [r.id for r in ijomc.reactions.query('EX_')]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()
df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])
df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

In [12]:
mu_meas = 0.74

F_TOL = 0.005

gap.reactions.BIOMASS_Ecoli_core_w_GAM.lower_bound = mu_meas*(1-F_TOL)
gap.reactions.BIOMASS_Ecoli_core_w_GAM.upper_bound = mu_meas*(1+F_TOL)

from cobra.solvers import gurobi_solver
from gurobipy import *

milp = gurobi_solver.create_problem(gap)
milp.ModelSense = GRB.MINIMIZE

In [13]:
milp.Params.IntFeasTol = 1e-9
milp.Params.OutputFlag = 1
milp.optimize()

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Optimize a model with 5545 rows, 2374 columns and 14186 nonzeros
Variable types: 1702 continuous, 672 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-07, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [7e-01, 1e+03]
  RHS range        [1e-03, 1e+03]
Presolve removed 1621 rows and 239 columns
Presolve time: 0.07s
Presolved: 3924 rows, 2135 columns, 10574 nonzeros
Variable types: 1477 continuous, 658 integer (658 binary)

Root relaxation: objective 6.442784e-01, 1313 iterations, 0.07 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.64428    0   10          -    0.64428      -     -    0s
     0     0    0.64428    0   10          -    0.64428      -     -    0s
     0     0    0.64428    0   10          -    0.64428      -     -    1s
     0     0   

In [14]:
#gap.optimize('minimize')
rxn_mu =gap.reactions.BIOMASS_Ecoli_core_w_GAM 
#muopt = rxn_mu
sol = gurobi_solver.format_solution(milp, gap)
muopt = sol.x_dict[rxn_mu.id]
yopt = [sol.x_dict[rxn.id] for rxn in gap.reactions.query('binary_')]
sum(yopt)
print('Initial mu_crowd=%g. Fitted within %g%%: %g <= %g <= %g' % (mu_crowd0, 100*F_TOL, rxn_mu.lower_bound, muopt, rxn_mu.upper_bound))
print('Number of non-zero binaries: %g' % sum(yopt))

Initial mu_crowd=0.873922. Fitted within 0.5%: 0.7363 <= 0.736653 <= 0.7437
Number of non-zero binaries: 4


In [15]:
# Fitted parameters
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol.x_dict[yid]
            if abs(y)>1e-10:
                print yid, y
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

binary_GND15 1.0
binary_GAPD16 1.0
binary_ENO16 1.0
binary_PFK16 1.0


In [16]:
len(gap.reactions.query('binary')) / len(kfit_dict)

14

In [17]:
len(var_cons_dict)

48

In [18]:
2018*0.7 / len(var_cons_dict)

29.429166666666664

In [19]:
[(k,v) for k,v in iteritems(kfit_dict) if abs(v)>1e-10]

[(u'GND', 1.7094017094017095e-05),
 (u'GAPD', 3.418803418803419e-05),
 (u'ENO', 3.418803418803419e-05),
 (u'PFK', 3.418803418803419e-05)]

### Plug back in to be sure

In [20]:
#----------------------------------------
# Starting from basal model
ijofit = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
crowding_bound = ijomc.metabolites.crowding._bound
crowding = Constraint('crowding')
crowding._bound = crowding_bound
crowding._constraint_sense = 'L'
for rid,kfit in iteritems(kfit_dict):
    rxn = ijofit.reactions.get_by_id(rid)
    rxn.add_metabolites({crowding:kfit})

In [21]:
ijofit.optimize()

<Solution 0.74 at 0x7f4d18d79dd0>

# Try using Benders

In [83]:
milp.ModelSense

1

In [ ]:
GRB.MINIMIZE

In [30]:
decomp = Decomposer(milp)

In [31]:
master,sub = decomp.benders_decomp()

Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter IntFeasTol to 1e-09
   Prev: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05


In [32]:
master.Params.presolve = 1
sub.Params.presolve = 1
master._verbosity = 1
master._precision_sub = 'dq'
master.optimize(cb_benders)

Changed value of parameter presolve to 1
   Prev: 0  Min: -1  Max: 2  Default: -1
Changed value of parameter presolve to 1
   Prev: -1  Min: -1  Max: 2  Default: -1
Optimize a model with 1 rows, 673 columns and 673 nonzeros
Variable types: 1 continuous, 672 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [0e+00, 0e+00]
Getting MINOS parameters...
Done in 1.19571 seconds with status 0
########################################
zmaster=672. zsub=2219.34. gap=-1547.34
########################################
Getting MINOS parameters...
Done in 0.685192 seconds with status 0
########################################
zmaster=0. zsub=2.87665. gap=-2.87665
########################################
Getting MINOS parameters...
Done in 0.307472 seconds with status 0
########################################
zmaster=0. zsub=2.87665. gap=-2.87665
########################################
G

Exception gurobipy.GurobiError: GurobiError() in 'gurobipy.callbackstub' ignored


Getting MINOS parameters...
Done in 0.59016 seconds with status 0
########################################
zmaster=1. zsub=3.87665. gap=-2.87665
########################################
Getting MINOS parameters...
Done in 0.614542 seconds with status 0
########################################
zmaster=1.2466. zsub=3.94682. gap=-2.70023
########################################
Getting MINOS parameters...
Done in 0.362694 seconds with status 0
########################################
zmaster=1.2466. zsub=3.94682. gap=-2.70023
########################################
Getting MINOS parameters...
Done in 0.433948 seconds with status 0
########################################
zmaster=1.2466. zsub=3.94682. gap=-2.70023
########################################
Getting MINOS parameters...
Done in 0.473922 seconds with status 0
########################################
zmaster=1.37375. zsub=3.68405. gap=-2.3103
########################################
Getting MINOS parameters...
Done in 0.533672 s

Exception gurobipy.GurobiError: GurobiError() in 'gurobipy.callbackstub' ignored


Getting MINOS parameters...
Done in 0.472474 seconds with status 0
########################################
zmaster=2.03129. zsub=3.51715. gap=-1.48586
########################################
Getting MINOS parameters...
Done in 0.431149 seconds with status 0
########################################
zmaster=2.15367. zsub=3.60053. gap=-1.44686
########################################
Getting MINOS parameters...
Done in 0.439977 seconds with status 0
########################################
zmaster=2.49955. zsub=4.99599. gap=-2.49644
########################################
Getting MINOS parameters...
Done in 0.364341 seconds with status 0
########################################
zmaster=2.51223. zsub=4.99063. gap=-2.4784
########################################
Getting MINOS parameters...
Done in 0.370752 seconds with status 0
########################################
zmaster=2.5519. zsub=5.0303. gap=-2.4784
########################################
Getting MINOS parameters...
Done in 0.3

...

Getting MINOS parameters...
Done in 0.413734 seconds with status 0
########################################
zmaster=2.45589. zsub=3.36711. gap=-0.91122
########################################
Getting MINOS parameters...
Done in 0.325534 seconds with status 0
########################################
zmaster=2.54809. zsub=5.04479. gap=-2.4967
########################################
Getting MINOS parameters...
Done in 0.552961 seconds with status 0
########################################
zmaster=2.55265. zsub=4.67047. gap=-2.11783
########################################
Getting MINOS parameters...
Done in 0.509856 seconds with status 0
########################################
zmaster=2.55265. zsub=3.83139. gap=-1.27875
########################################
Getting MINOS parameters...
Done in 0.383097 seconds with status 0
########################################
zmaster=2.55265. zsub=3.83139. gap=-1.27875
########################################
Getting MINOS parameters...
Done in 

Exception gurobipy.GurobiError: GurobiError() in 'gurobipy.callbackstub' ignored


Getting MINOS parameters...
Done in 0.445056 seconds with status 0
########################################
zmaster=2.40757. zsub=4.27043. gap=-1.86286
########################################
Getting MINOS parameters...
Done in 0.344934 seconds with status 0
########################################
zmaster=2.40757. zsub=4.27043. gap=-1.86286
########################################
Getting MINOS parameters...
Done in 0.3246 seconds with status 0
########################################
zmaster=2.42439. zsub=4.28724. gap=-1.86286
########################################
Getting MINOS parameters...
Done in 0.332363 seconds with status 0
########################################
zmaster=2.50863. zsub=4.37149. gap=-1.86286
########################################
Getting MINOS parameters...
Done in 0.324456 seconds with status 0
########################################
zmaster=2.50863. zsub=4.37149. gap=-1.86286
########################################
Getting MINOS parameters...
Done in 0

Exception gurobipy.GurobiError: GurobiError() in 'gurobipy.callbackstub' ignored


Getting MINOS parameters...
Done in 0.471689 seconds with status 0
########################################
zmaster=2.53723. zsub=4.66834. gap=-2.13112
########################################
Getting MINOS parameters...
Done in 0.42743 seconds with status 0
########################################
zmaster=2.55831. zsub=4.63536. gap=-2.07704
########################################
Getting MINOS parameters...
Done in 0.398903 seconds with status 0
########################################
zmaster=2.66335. zsub=4.97903. gap=-2.31568
########################################
Getting MINOS parameters...
Done in 0.386224 seconds with status 0
########################################
zmaster=2.71413. zsub=5.13016. gap=-2.41603
########################################
Getting MINOS parameters...
Done in 0.392566 seconds with status 0
########################################
zmaster=2.71323. zsub=5.02891. gap=-2.31568
########################################
Getting MINOS parameters...
Done in 

In [33]:
master.ObjVal

2.876645084621373

In [89]:
sub.ObjVal

2.8766450846777474

In [80]:
yoptB = np.array([y.X for y in decomp._ys])
sum(yoptB)

0.0

In [85]:
decomp._ys

[<gurobi.Var 358 (value -0.0)>,
 <gurobi.Var 361 (value -0.0)>,
 <gurobi.Var 364 (value -0.0)>,
 <gurobi.Var 367 (value -0.0)>,
 <gurobi.Var 370 (value -0.0)>,
 <gurobi.Var 373 (value -0.0)>,
 <gurobi.Var 376 (value -0.0)>,
 <gurobi.Var 379 (value -0.0)>,
 <gurobi.Var 382 (value -0.0)>,
 <gurobi.Var 385 (value -0.0)>,
 <gurobi.Var 388 (value -0.0)>,
 <gurobi.Var 391 (value -0.0)>,
 <gurobi.Var 394 (value -0.0)>,
 <gurobi.Var 397 (value -0.0)>,
 <gurobi.Var 400 (value -0.0)>,
 <gurobi.Var 403 (value -0.0)>,
 <gurobi.Var 406 (value -0.0)>,
 <gurobi.Var 409 (value -0.0)>,
 <gurobi.Var 412 (value -0.0)>,
 <gurobi.Var 415 (value -0.0)>,
 <gurobi.Var 418 (value -0.0)>,
 <gurobi.Var 421 (value -0.0)>,
 <gurobi.Var 424 (value -0.0)>,
 <gurobi.Var 427 (value -0.0)>,
 <gurobi.Var 430 (value -0.0)>,
 <gurobi.Var 433 (value -0.0)>,
 <gurobi.Var 436 (value -0.0)>,
 <gurobi.Var 439 (value -0.0)>,
 <gurobi.Var 442 (value -0.0)>,
 <gurobi.Var 445 (value -0.0)>,
 <gurobi.Var 448 (value -0.0)>,
 <gurobi

In [69]:
mu_ind = gap.reactions.index(gap.reactions.BIOMASS_Ecoli_core_w_GAM)

In [84]:
cons_mu = sub.model.getConstrByName(str(mu_ind))
cons_mu.Pi

AttributeError: Unable to retrieve attribute 'Pi'